In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Carga y preprocesamiento de datos
df = pd.read_csv('/content/Indicadores_municipales_sabana_DA (1).csv', encoding='ISO-8859-1')
df_procesado = df.drop(columns=['nom_mun', 'nom_ent'])

# Valores promedio de categorías
target_column = 'pobreza_alim_00'

# Rellenamos los valores faltantes con los promedios
promedio_global = np.nanmean(df_procesado[target_column])
df_procesado[target_column] = df_procesado[target_column].fillna(promedio_global)

# Identificar columnas categóricas
columnas_categoricas = df_procesado.select_dtypes(include=['object']).columns

# Construir un transformador para aplicar codificación one-hot a columnas categóricas
transformador = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), columnas_categoricas)
    ],
    remainder='passthrough'
)

# Definir el modelo de regresión lineal con la codificación one-hot y manejo de valores faltantes
modelo_regresion_lineal = Pipeline(steps=[
    ('transformador', transformador),
    ('imputador', SimpleImputer(strategy='mean', missing_values=np.nan)),  # Asegúrate de especificar missing_values
    ('regresion_lineal', LinearRegression())
])

# Define el DataFrame eliminando la columna a predecir
# Matriz de características X y la variable objetivo y
X = df_procesado.drop(columns=[target_column])
y = df_procesado[target_column]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar el modelo
modelo_regresion_lineal.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_regresion_lineal = modelo_regresion_lineal.predict(X_test)

# Calcular la precisión del modelo
precision_regresion_lineal = metrics.r2_score(y_test, y_pred_regresion_lineal)
print(f"Precisión del modelo de regresión lineal: {precision_regresion_lineal}")

# Mostrar las últimas diez reales y predictivas
ultimas_diez_real = df_procesado[target_column].iloc[-10:]
indices_ultimas_diez = ultimas_diez_real.index

# Eliminar y adecuar
X_ultimas_diez = df_procesado.loc[indices_ultimas_diez].drop(columns=[target_column])
y_pred_regresion_lineal_ultimas_diez = modelo_regresion_lineal.predict(X_ultimas_diez)

# Mostrar
df_resultados_ultimas_diez = pd.DataFrame({
    'Real (pobreza_alim_00)': ultimas_diez_real.values,
    'Regresión Lineal': y_pred_regresion_lineal_ultimas_diez
}, index=indices_ultimas_diez)

print(df_resultados_ultimas_diez)


Precisión del modelo de regresión lineal: 0.999526811174915
      Real (pobreza_alim_00)  Regresión Lineal
2446                    34.1         33.749723
2447                    22.8         23.034222
2448                    39.2         39.207932
2449                    27.7         28.276508
2450                    35.5         35.604440
2451                    47.9         47.803921
2452                    20.0         20.108638
2453                    14.2         14.497776
2454                    29.4         29.778273
2455                    37.2         37.447396
